# Introduction

For this example we will take the AutoEncoder model trained in the notebook `Example Model Development.ipynb` and go through the steps which would be needed to deploy it into a production environment in the cloud using the command line tool `composer` which is designed to move a data science model from a development environment such as jupyterhub or on a laptop to a cloud tenant. The cloud tenant will have new data streaming in from remote locations so that we can use our model to make predictions on those data in real time.

# Arundo Fabric

For the workshop we have held back some of the data and are now streaming it into Arundo's industrial enterprise cloud environment Fabric. You can access Fabric by navigating to https://sand01.arundo.com and using the login credentials provided at the workshop (ask if you don't know the credentials!) Note that we recommend to use Chrome as your browser  although others should work too.

When logged in you will see a screen like this:

<img src="data/images/fabric.png" width=85% />

The key components you will use are the Models Manager and the Pipelines Manager. 

## Models Manager

When you deploy the model its status will be visible in the Models Manager, you can also see recent logs for your model there in case it does not seem to be performing as you expect or the deployment fails. Feel free to look at the model which has already been deployed with name `AutoEncoder_drillxgit` by clicking on its name in the table:

<img src="data/images/fabric_models.png" width=85% />
<img src="data/images/deployed_model.png" width=85% />

Don't worry about the `404` responses you see for `GET metrics/` that functionality has not been setup for this workshop so that's expected.

Return to the model list by hitting the back button on your browser, now click on the `Use` box for that model. You will need to login again with the same credentials; this is because the model lives at a standalone url to the Fabric tenant but has been assigned the same user permissions so only users of the tenant can access it.

When logged in you will see a screen like this:

<img src="data/images/model_landing.png" width=85% />

If you navigate to the POST link for endpoint `stream_prediction` you will see an autorendered web form:

<img src="data/images/endpoint1a.png" style="display:inline" width=48% />
<img src="data/images/endpoint1b.png" style="display:inline" width=48% />

This is not particularly useful as to make a prediction you have to enter all 29 input values by hand and hit `Execute Model`; however, the purpose of this endpoint is not to be interactive but to automatically make predictions on a stream of data. You will go through the steps of setting this up and will see what that looks like in the Pipelines Manager section below.

If you navigate to the POST link for the endpoint `calculate_score` you will see a different autoredered web form. Leave the box blanks and hit `Execute Model`, this will generate a tab showing the current model score and a plot of the confusion matrix of the hidden data. Don't worry that the score is 0.0, it just used some toy data, we will give the SAS token for the real hidden data during the workshop; if you think you are ready to score your model please ask for the SAS token.

<img src="data/images/endpoint2.png" width=85% />

## Pipelines Manager

If you now navigate to the Pipelines Manager in Fabric you will see a page like this:

<img src="data/images/pipeline.png" width=85% />

Click on the name of the example deployed pipeline `autoencoder_driixgit`, after it has loaded you will see the data streaming in on the left, one plot for each of the 29 sensor variables. On the right you will see the model prediction for the `stream_prediction` endpoint we looked at above which we have used `composer` to attach to the pipeline. Note that the version of Pipelines Manager we are using for this workshop only supports one model output.

<img src="data/images/streaming_pipeline.png" width=85% />


# Arundo Composer

We will now go through the steps needed to deploy a model to Fabric using the Composer command line tool. You might find it useful to open a terminal from the your notebook homepage, but we can also issue command line calls directly from this notebook using the `!` prefix. The environment you are working in already has Composer installed. First let's briefly explore the kind of functionality Composer offers:

In [1]:
!composer --help

Usage: composer [OPTIONS] COMMAND [ARGS]...

  A CLI tool for Arundo Fabric

Options:
  --debug / --no-debug
  --local / --no-local
  --help                Show this message and exit.

Commands:
  bundle          Bundle up a model for publishing
  check           See what your current settings are
  clear_password  Clears a saved password
  config          See your config file
  container       Container management commands
  dev             Arundo developer only commands
  extensions      Extension management commands
  file            File management commands
  init            Create a new model skeleton
  login           Login to Arundo Fabric
  logout          Log out of Fabric
  model           Model management commands
  pipeline        How pipelines are managed
  publish         Publish a model to Fabric
  refresh         Ensure that your access token is refreshed
  request         Make a curl-style request with auth headers
  run             Run a model locally for testing
  se

## Connecting to Fabric

There are a wide range of commands a data scientist can use to develop and deploy their models. Next we want to get composer connected to the cloud tenant, as the current settings are not yet connected. The credentials you have for this workshop are for a Fabric tenant in environment `sand01` so we need to connect Composer to that environment:

In [2]:
!composer switch sand01
!composer check

You've switched to a new server (sand01) and will have to login again
Your settings have been saved.
Composer Version:   0.2.1
Email:              
Saved Password:     False
Server:             https://sand01.arundo.com
Access Remaining:   None
Refresh Remaining:  None
OS Version:         linux


We also need to set the email to match the Fabric credentials, this is done as follows:

In [3]:
!composer set email <MY_USER_EMAIL>

Now we connect to the cloud tenant and obtain the authorization to interact with it. If you are working with the terminal you can run `$ composer login` (make sure you are in the directory `work`) and enter the password for your user email. This command won't work from the notebook as it requires user input at the command line which jupyter doesn't support). If you prefer to stay in the notebook you can ask one of the workshop conveners to provide you with an authorization token which you can then use in the following command to authorize and connect to the tenant environement:

In [4]:
!composer set token <MY_AUTH_TOKEN>

Set your token!


You should now be connected which you can confirm by seeing that the field `Access Remaining` is no longer `None` but has some finite time remaining. If that's not the case please ask for help.

In [5]:
!composer check

Composer Version:   0.2.1
Email:              user_amld@drillx.com
Saved Password:     False
Server:             https://sand01.arundo.com
Access Remaining:   5h59m
Refresh Remaining:  None
OS Version:         linux


## Preparing for Deployment

Now that we have composer connected to the Fabric tenant we need to construct an `app.py` using the Arundo Python Model Runtime libraries which provide a user friendly way to generate the web interface endpoints you saw above  - essentially all of the complicated stuff is taken care of in decorators!

To deploy to the cloud you will need three files:
```
app.py
config.yaml
requirements.txt```
In fact `requirements.txt` is not even necessary if you have no additional Python library dependencies beyond the standard libraries. The following cells show the corresponding contents of these files for the model which you have already seen deployed, executing a given cell will overwrite the corresponding file in your notebook directory due to the `%%writefile <FILENAME>` header, so you can use these cells to edit the files you will deploy with your model.

### config.yaml

This file names the model and provides some basic version control. Note that in the tenant no two models can have the same name and version number regardless of which user deployed it. If you try to deploy a model whose name/version already exists the build will fail. To make sure your name does not clash with other workshop users we suggest to add your github user handle in the model name, this will also make it easier to find in the Models Management table when you have deployed to Fabric. If your build fails, fix the code and bump the version numer when redeploying.

In [6]:
%%writefile config.yaml

model_name: AutoEncoder-drillxgit
model_version: 0.0.1
wrapper_version: 2

Overwriting config.yaml


### requirements.txt

Here you should add all the Python library requirements needed to execute your model so that they are installed correctly in the deployed environment. The main cause of failures in the deployment process tends to be a missing entry in this file, particularly of hidden dependencies. If your model has associated non-python binary dependencies which would need e.g. `apt-get install <PACKAGE>` then you will need an extra file `dependencies.apt` which lists these. For most workflows that shouldn't be needed.

In [7]:
%%writefile requirements.txt
# needed by the model
Keras==2.1.3
tensorflow==1.4.1
h5py==2.7.1

# needed by the endpoint functions
numpy==1.13.1
scikit-learn==0.19.1
matplotlib==2.1.2
seaborn==0.8.1
azure-storage==0.36.0

Overwriting requirements.txt


### app.py

This file is where most of the work is done. Here there have been two endpoints implemented, the details of which are given in the docstrings for those endpoints. To update this functionality to support your own model you should implement the functions `row_prediction` and `vectorized_prediction` following the example shown for the trained autoencode.

In [8]:
%%writefile app.py

"""
To deploy your own serialized model for deployment to a stream of live data
and to calculate the score of your model on the hidden data implement
the functions:

row_prediction which takes a list as input in the order:
[Vol, S1, S2, ..., S27, S28];

vectorized_prediction which takes a np.ndarry as input with features as
columns in the same order and entries as rows.
"""
# these imports are needed for the endpoints which you do not need to change
from runtime.framework import (endpoint, argument, returns, visualization)
from runtime.schema import fields
import numpy as np
from sklearn.metrics import mean_squared_error
from amld_functions import (get_data, get_score, plot_confmat)

# model dependent libraries
from keras.models import load_model
import keras.backend as K
from sklearn.externals import joblib


# global variables can be loaded into memory at deployment to improve efficiency
# be careful if using tensorflow globally
g_scaler = joblib.load('scaler.model')
g_threshold99 = 3.8257


def row_prediction(row):
    """
    This function should return the prediction of your model on a single row
    of data.
    Inputs:
    param row: np.array shape (, 29) with feature order
               [Vol, S1, S2, ..., S27, S28]
    Returns:
    <bool>: True for predicted anomaly, False otherwise
    """
    # apply the globally loaded feature scaler
    scaled = g_scaler.transform(np.atleast_2d(row))
    # load the trained autoencoder model
    ae_model = load_model('model.h5')
    # get predicted autoencoder output for the scaled input row
    predicted = ae_model.predict(scaled)
    # tidy up the Keras backend for this call
    K.clear_session()
    # calculate the mean squared error for this row
    mse = mean_squared_error(predicted.T, scaled.T)
    # return True if the mse is over the 99% threshold for non-anomalous data
    return (mse > g_threshold99)


def vectorized_prediction(matrix):
    """
    This function should return a vector of predictions of your model on an
    input matrix of batch data with m examples.
    Inputs:
    param matrix: np.ndarray shape (m, 29) with feature order
                  [Vol, S1, S2, ..., S27, S28]
    Returns:
    <np.ndarray>: shape (m, 1) with m_i True for predicted anomaly and False
                  otherwise
    """
    # apply the globally loaded feature scaler
    scaled = g_scaler.transform(matrix)
    # load the trained autoencoder model
    ae_model = load_model('model.h5')
    # get predicted autoencoder output for the scaled input batch data
    predicted = ae_model.predict(scaled)
    # calculate the mean squared error for all examples
    mse = mean_squared_error(predicted.T, scaled.T, multioutput='raw_values')
    # return vector with True if the mse is over the 99% threshold
    return (mse > g_threshold99)


###
# You do not need to edit below this line but are encouraged to understand
# the implementation of the endpoints, just ask if you want more information
# about endpoint implementation and functionality!
###


@endpoint()
@argument("Vol", type=float, description="Value of input sensor Vol")
@argument("S1", type=float, description="Value of input sensor S1")
@argument("S2", type=float, description="Value of input sensor S2")
@argument("S3", type=float, description="Value of input sensor S3")
@argument("S4", type=float, description="Value of input sensor S4")
@argument("S5", type=float, description="Value of input sensor S5")
@argument("S6", type=float, description="Value of input sensor S6")
@argument("S7", type=float, description="Value of input sensor S7")
@argument("S8", type=float, description="Value of input sensor S8")
@argument("S9", type=float, description="Value of input sensor S9")
@argument("S10", type=float, description="Value of input sensor S10")
@argument("S11", type=float, description="Value of input sensor S11")
@argument("S12", type=float, description="Value of input sensor S12")
@argument("S13", type=float, description="Value of input sensor S13")
@argument("S14", type=float, description="Value of input sensor S14")
@argument("S15", type=float, description="Value of input sensor S15")
@argument("S16", type=float, description="Value of input sensor S16")
@argument("S17", type=float, description="Value of input sensor S17")
@argument("S18", type=float, description="Value of input sensor S18")
@argument("S19", type=float, description="Value of input sensor S19")
@argument("S20", type=float, description="Value of input sensor S20")
@argument("S21", type=float, description="Value of input sensor S21")
@argument("S22", type=float, description="Value of input sensor S22")
@argument("S23", type=float, description="Value of input sensor S23")
@argument("S24", type=float, description="Value of input sensor S24")
@argument("S25", type=float, description="Value of input sensor S25")
@argument("S26", type=float, description="Value of input sensor S26")
@argument("S27", type=float, description="Value of input sensor S27")
@argument("S28", type=float, description="Value of input sensor S28")
@returns("predicted_class", type=float,
         description="Prediction as a float: 1.0 if anomaly, 0.0 otherwise")
def stream_prediction(
    Vol,
    S1,  S2,  S3,  S4,  S5,  S6,  S7,  S8,  S9,  S10,
    S11, S12, S13, S14, S15, S16, S17, S18, S19, S20,
    S21, S22, S23, S24, S25, S26, S27, S28
):
    """
    This endpoint is used to connect a live stream of data to a model and
    return predictions in real time for each new feature row as it arrives
    from a remote location.
    """
    # take the inputs and put them into and ordered np.array shape (, 29)
    feature_list = np.array([
        Vol,
        S1,  S2,  S3,  S4,  S5,  S6,  S7,  S8,  S9,  S10,
        S11, S12, S13, S14, S15, S16, S17, S18, S19, S20,
        S21, S22, S23, S24, S25, S26, S27, S28
    ])
    # return the predicted output from the function row_prediction
    return float(row_prediction(feature_list))


@endpoint()
@argument("sas_token", type=str,
          description="To score your model with the hidden data enter the "
                      "sas_token provided by workshop conveners, not needed "
                      "for local test", default="")
@returns("score", type=float, description="The score of the deployed model "
                                           "on the hidden data")
@visualization(tab='confusion_matrix.html')
def calculate_score(sas_token):
    """
    This endpoint is used to load some hidden batch data from the cloud and
    calculate the score of the deployed model. It will also display the
    confusion matrix of the model in a tab on the model execution page.
    """
    # get the input data to calculate the score
    X, y = get_data(sas_token)
    # get the prediction vector from the function vectorized_prediction
    prediction = vectorized_prediction(X)
    # convert the predicted data to the same format as the hidden targets
    y_pred = ['normal' if not ipred else 'fraud' for ipred in prediction]
    # plot and save the confusion matrix for display in the model page
    plot_confmat(y, y_pred)
    # return the model's score and the name of the saved plot to render
    return get_score(y, y_pred)

Overwriting app.py


## Testing Before Deployment

To test if `app.py` is working as expected we can run the model in the current environment before deploying. Note that if this was on your laptop you could also access the auto rendered web form by navigating to port 5000 on the local host. The VM has not been setup to render the forms so we will instead check by sending a request from the command line to that localhost port.

First you will need to open a terminal from the notebook homepage and issue the command `composer run`, if this fails you will need to debug `app.py`. Otherwise you should be able to execute the two following commands:

In [9]:
!curl -d '@test_row.json' -H "Content-Type: application/json" http://localhost:5000/stream_prediction 

{
  "predicted_class": 0.0
}


In [10]:
!curl -d '{"sas_token": ""}' -H "Content-Type: application/json" http://localhost:5000/calculate_score

{
  "score": 0.0
}


## Deploying to Fabric

Deployment is simple:

In [11]:
!composer publish

Validating model...
{'model_name': 'AutoEncoder-drillxgit', 'model_version': '0.0.1', 'wrapper_version': 2}
Model validated!
Bundling model...
Bundling app in folder /home/jovyan/work
Making zip file: work.zip
Model bundled!
{'model_name': 'AutoEncoder-drillxgit', 'model_version': '0.0.1', 'wrapper_version': 2}
Creating model doc...
Model doc created id = 3dcf0eed-312d-d09a-1a3f-466370b08520
Uploading model zip file...
Model upload succeeded
Build triggered
Build status is: queued after 1m19s 
Build status is: building after 3m52s 
Build status is: uploading after 4m31s 
Build status is: deploying after 5m51s 
Build status is: completed after 5m56s 
Model ID:         3dcf0eed-312d-d09a-1a3f-466370b08520
Name:             AutoEncoder-drillxgit
Version:          0.0.1
Wrapper Version:  2
Build Status:     completed
Owner:            user_amld@drillx.com
Created:          2018-01-26 22:35:57.712000
Scale:            1/1
Base URL:         https://sand01.arundo.com/model/3dcf0eed-312d-d09a-

If the build or deploy fails, see if you can determine why or call one of the session conveners to help you debug. Otherwise you will now see the model listed as follows:

In [12]:
!composer model list

Model ID                              Name                   Version    Owner                   W.V.  Build Status
------------------------------------  ---------------------  ---------  --------------------  ------  --------------
3dcf0eed-312d-d09a-1a3f-466370b08520  AutoEncoder-drillxgit  0.0.1      user_amld@drillx.com       2  completed
f4c3b095-4f69-e088-ba65-7a5242e1add2  AutoEncoder_drillxgit  0.0.1      user_amld@drillx.com       2  completed


Further information about the deployed model can be obtained directly from composer, for example:

In [13]:
!composer model details <MODEL_UUID>

Model ID:         3dcf0eed-312d-d09a-1a3f-466370b08520
Name:             AutoEncoder-drillxgit
Version:          0.0.1
Wrapper Version:  2
Build Status:     completed
Owner:            user_amld@drillx.com
Created:          2018-01-26 22:35:57.712000
Scale:            1/1
Base URL:         https://sand01.arundo.com/model/3dcf0eed-312d-d09a-1a3f-466370b08520


In [14]:
!composer model endpoints <MODEL_UUID>

  Endpoints
  ├── calculate_score
  │   ├── Inputs
  │   │   └── sas_token: <class 'str'>
  │   └── Outputs
  │       └── score: <class 'float'>
  └── stream_prediction
      ├── Inputs
      │   ├── S1: <class 'float'>
      │   ├── S10: <class 'float'>
      │   ├── S11: <class 'float'>
      │   ├── S12: <class 'float'>
      │   ├── S13: <class 'float'>
      │   ├── S14: <class 'float'>
      │   ├── S15: <class 'float'>
      │   ├── S16: <class 'float'>
      │   ├── S17: <class 'float'>
      │   ├── S18: <class 'float'>
      │   ├── S19: <class 'float'>
      │   ├── S2: <class 'float'>
      │   ├── S20: <class 'float'>
      │   ├── S21: <class 'float'>
      │   ├── S22: <class 'float'>
      │   ├── S23: <class 'float'>
      │   ├── S24: <class 'float'>
      │   ├── S25: <class 'float'>
      │   ├── S26: <class 'float'>
      │   ├── S27: <class 'float'>
      │   ├── S28: <class 'float'>
      │   ├── S3: <class 'float'>
      │   ├── S4: <class 'float'>
      │   ├──

## Creating a Streaming Pipeline

Next we would like to create a pipeline to expose the data streaming into the tenant and then deploy the `stream_prediction` endpoint to that pipeline. That is very straight forward, first create the pipeline:

In [19]:
!composer pipeline create -n autoencoder_drillx_git

Successfully created pipeline (id=7f52c8b2-eed4-3412-933a-2f7e57e2d9fc).


In [20]:
!composer pipeline list

Pipeline ID                           Pipeline Name           Owner                 Created Date
------------------------------------  ----------------------  --------------------  --------------------------
00d1ccfe-7b0b-e41e-c7c3-d3f752910488  autoencoder_drillxgit   user_amld@drillx.com  2018-01-25 22:35:46.590000
7f52c8b2-eed4-3412-933a-2f7e57e2d9fc  autoencoder_drillx_git  user_amld@drillx.com  2018-01-26 22:47:17.545000


In [21]:
!composer pipeline details <PIPELINE_UUID>

           +---------------+
---------- | Pipeline Info | ----------
           +---------------+
Pipeline ID:    7f52c8b2-eed4-3412-933a-2f7e57e2d9fc
Pipeline Name:  autoencoder_drillx_git
Created By:     user_amld@drillx.com
Created Date:   2018-01-26 22:47:17.545000

           +---------------+
---------- | Model Details | ----------
           +---------------+
           +------------------+
---------- | Pipeline Diagram | ----------
           +------------------+
  autoencoder_drillx_git


## Adding a Model to a Pipeline

Now we add the required model endpoint. Note that because we have a naming clash between the incoming sensors and the inputs defined in the endpoint we need to define a file `mapping.yaml` which defines that mapping:

In [23]:
%%writefile mapping.yaml
Vol: demo.fraud_amld.live_data_raw.Vol
S1: demo.fraud_amld.live_data_raw.S1
S2: demo.fraud_amld.live_data_raw.S2
S3: demo.fraud_amld.live_data_raw.S3
S4: demo.fraud_amld.live_data_raw.S4
S5: demo.fraud_amld.live_data_raw.S5
S6: demo.fraud_amld.live_data_raw.S6
S7: demo.fraud_amld.live_data_raw.S7
S8: demo.fraud_amld.live_data_raw.S8
S9: demo.fraud_amld.live_data_raw.S9
S10: demo.fraud_amld.live_data_raw.S10
S11: demo.fraud_amld.live_data_raw.S11
S12: demo.fraud_amld.live_data_raw.S12
S13: demo.fraud_amld.live_data_raw.S13
S14: demo.fraud_amld.live_data_raw.S14
S15: demo.fraud_amld.live_data_raw.S15
S16: demo.fraud_amld.live_data_raw.S16
S17: demo.fraud_amld.live_data_raw.S17
S18: demo.fraud_amld.live_data_raw.S18
S19: demo.fraud_amld.live_data_raw.S19
S20: demo.fraud_amld.live_data_raw.S20
S21: demo.fraud_amld.live_data_raw.S21
S22: demo.fraud_amld.live_data_raw.S22
S23: demo.fraud_amld.live_data_raw.S23
S24: demo.fraud_amld.live_data_raw.S24
S25: demo.fraud_amld.live_data_raw.S25
S26: demo.fraud_amld.live_data_raw.S26
S27: demo.fraud_amld.live_data_raw.S27
S28: demo.fraud_amld.live_data_raw.S28

Overwriting mapping.yaml


Now we put it all together:

In [24]:
!composer pipeline add_model -e stream_prediction -mf mapping.yaml <PIPELINE_UUID> <MODEL_UUID>

Endpoint 'stream_prediction' input/output name map:

| Group   | Model Item      | Pipeline Mapped Item                          |
|:--------|:----------------|:----------------------------------------------|
| input   | S1              | demo.fraud_amld.live_data_raw.S1              |
| input   | S10             | demo.fraud_amld.live_data_raw.S10             |
| input   | S11             | demo.fraud_amld.live_data_raw.S11             |
| input   | S12             | demo.fraud_amld.live_data_raw.S12             |
| input   | S13             | demo.fraud_amld.live_data_raw.S13             |
| input   | S14             | demo.fraud_amld.live_data_raw.S14             |
| input   | S15             | demo.fraud_amld.live_data_raw.S15             |
| input   | S16             | demo.fraud_amld.live_data_raw.S16             |
| input   | S17             | demo.fraud_amld.live_data_raw.S17             |
| input   | S18             | demo.fraud_amld.live_data_raw.S18             |
| input   |

In [25]:
!composer pipeline details <PIPELINE_UUID>

           +---------------+
---------- | Pipeline Info | ----------
           +---------------+
Pipeline ID:    7f52c8b2-eed4-3412-933a-2f7e57e2d9fc
Pipeline Name:  autoencoder_drillx_git
Created By:     user_amld@drillx.com
Created Date:   2018-01-26 22:47:17.545000

           +---------------+
---------- | Model Details | ----------
           +---------------+
Model ID:      3dcf0eed-312d-d09a-1a3f-466370b08520
Name:          AutoEncoder-drillxgit-0.0.1-stream_prediction
Version:       0.3
Url:           https://sand01.arundo.com/model/3dcf0eed-312d-d09a-1a3f-466370b08520/stream_prediction
Inputs:        ['S1', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S2', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'Vol']
Outputs:       ['predicted_class']
Created By:    user_amld@drillx.com
Created Date:  2018-01-26 22:41:56.290000

           +------------------+
---------- | Pipeline Diagram | ---------

Finally we can delete the pipeline and model we created:

In [27]:
!composer pipeline delete <PIPELINE_UUID>

Successfully deleted pipeline (id=7f52c8b2-eed4-3412-933a-2f7e57e2d9fc).


In [28]:
!composer pipeline list

Pipeline ID                           Pipeline Name          Owner                 Created Date
------------------------------------  ---------------------  --------------------  --------------------------
00d1ccfe-7b0b-e41e-c7c3-d3f752910488  autoencoder_drillxgit  user_amld@drillx.com  2018-01-25 22:35:46.590000


In [29]:
!composer model delete <MODEL_UUID>

Deleting model 3dcf0eed-312d-d09a-1a3f-466370b08520
Deleted your model 3dcf0eed-312d-d09a-1a3f-466370b08520


In [30]:
!composer model list

Model ID                              Name                   Version    Owner                   W.V.  Build Status
------------------------------------  ---------------------  ---------  --------------------  ------  --------------
f4c3b095-4f69-e088-ba65-7a5242e1add2  AutoEncoder_drillxgit  0.0.1      user_amld@drillx.com       2  completed
